<a href="https://colab.research.google.com/github/Wilsonmfreire/Treinando_Spark_com_Pyspark/blob/main/Spark_Udemy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Instalando o Java
!apt-get install openjdk-8-jdk-headless -qq >/dev/null

In [2]:
#Baixar a versão mais recente do spark
!wget -q https://dlcdn.apache.org/spark/spark-3.2.0/spark-3.2.0-bin-hadoop3.2.tgz

In [3]:
#Dezipar spark
!tar xf /content/spark-3.2.0-bin-hadoop3.2.tgz

In [5]:
#Criar variável de ambiente
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64/"
os.environ["SPARK_HOME"] = "/content/spark-3.2.0-bin-hadoop3.2/"

In [4]:
#instalar a lib findspark que ajuda a localizar o Spark no sistema e importá-lo como uma biblioteca regulgar
!pip install -q findspark

In [6]:
#import a lib findspark
import findspark
findspark.init()

In [7]:
#criar spark session
#StructField
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import SQLContext
from pyspark.sql.functions import * 
from pyspark.sql import functions as Fun

spark = SparkSession.builder\
        .master('local')\
        .appName('sparkcolab')\
        .getOrCreate()

In [ ]:
#Upload de arquivos
from google.colab import files
arquivo = files.upload()

In [8]:
!wget www.datascientist.com.br/bigdata/download.zip

--2021-12-13 10:43:30--  http://www.datascientist.com.br/bigdata/download.zip
Resolving www.datascientist.com.br (www.datascientist.com.br)... 108.179.192.203
Connecting to www.datascientist.com.br (www.datascientist.com.br)|108.179.192.203|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 207452 (203K) [application/zip]
Saving to: ‘download.zip’

download.zip        100%[===================>] 202.59K  1.30MB/s    in 0.2s    

2021-12-13 10:43:31 (1.30 MB/s) - ‘download.zip’ saved [207452/207452]



In [9]:
!unzip /content/download.zip

Archive:  /content/download.zip
   creating: download/Atividades/
  inflating: download/Atividades/Clientes.parquet  
  inflating: download/Atividades/ItensVendas.parquet  
  inflating: download/Atividades/Produtos.parquet  
  inflating: download/Atividades/Vendas.parquet  
  inflating: download/Atividades/Vendedores.parquet  
  inflating: download/Carros.csv     
  inflating: download/Churn.csv      
   creating: download/demo/
  inflating: download/demo/1.CreateTable.sql  
  inflating: download/demo/2.InsertClientes.sql  
  inflating: download/demo/3.InsertIntoProdutos.sql  
  inflating: download/demo/4.InsertIntoVendedores.sql  
  inflating: download/demo/5.InsertIntoVendas.sql  
  inflating: download/demo/6.InsertItensVenda.sql  
  inflating: download/despachantes.csv  
  inflating: download/iris.csv       
   creating: download/mongo/
  inflating: download/mongo/posts.json  
  inflating: download/reclamacoes.csv  
   creating: download/streaming/
  inflating: download/streaming/1.

In [11]:
#Criando primeiro dataFrame
df1 = spark.createDataFrame([("Pedro", 10),("Maria", 20),("Jose",40)])
df1.show( )

+-----+---+
|   _1| _2|
+-----+---+
|Pedro| 10|
|Maria| 20|
| Jose| 40|
+-----+---+



In [ ]:
#criando dataFrame com StructField
#nao entendi pq passar o data no createDaaFrame deu certo
dados = [("1", "Pedro"), 
      ("2", "Maria")
    ]

schema = StructType([ \
    StructField("id", StringType(), True),\
    StructField("nome", StringType(), True),\
  ])

df2 = spark.createDataFrame(data=data2,schema=schema)
df2.printSchema()
df2.show()

In [13]:
#criando dataFrame sem StructField
schema2 = "Produtos STRING, Vendas INT"
vendas = [["Caneta", 10], ["Lápis", 20], ["Caneta", 40]]
df3 = spark.createDataFrame(vendas, schema2)
df3.show()

+--------+------+
|Produtos|Vendas|
+--------+------+
|  Caneta|    10|
|   Lápis|    20|
|  Caneta|    40|
+--------+------+



In [14]:
#trabalhando com sum
#precisamos agrupar o que não iremos somar e usar .agg(sum("passar coluna com valores"))
agrupados = df3.groupBy("Produtos").agg(sum("vendas"))
agrupados.show()

+--------+-----------+
|Produtos|sum(vendas)|
+--------+-----------+
|  Caneta|         50|
|   Lápis|         20|
+--------+-----------+



In [15]:
#passando o dataFrame direto sem associar ao um novo
df3.groupBy("Produtos").agg(sum("Vendas")).show()

+--------+-----------+
|Produtos|sum(Vendas)|
+--------+-----------+
|  Caneta|         50|
|   Lápis|         20|
+--------+-----------+



In [16]:
# Trabalhando com select e expressões
df3.select('Vendas').show()
df3.select('Produtos',\
           'Vendas',\
           expr('Vendas *0.2')).show()

+------+
|Vendas|
+------+
|    10|
|    20|
|    40|
+------+

+--------+------+--------------+
|Produtos|Vendas|(Vendas * 0.2)|
+--------+------+--------------+
|  Caneta|    10|           2.0|
|   Lápis|    20|           4.0|
|  Caneta|    40|           8.0|
+--------+------+--------------+



In [17]:
#Trabalhando com exp e select
df3.select("Vendas").show()
df3.select("Produtos", "Vendas", expr("Vendas *0.2")).show()

+------+
|Vendas|
+------+
|    10|
|    20|
|    40|
+------+

+--------+------+--------------+
|Produtos|Vendas|(Vendas * 0.2)|
+--------+------+--------------+
|  Caneta|    10|           2.0|
|   Lápis|    20|           4.0|
|  Caneta|    40|           8.0|
+--------+------+--------------+



In [18]:
#Atribuindo um schema
#Ler arquivo csv
arqschema = "id INT, nome STRING, status STRING, cidade STRING, vendas INT, data STRING"
despachantes = spark.read.csv(
    "/content/download/despachantes.csv",\
    header=False,\
    schema= arqschema\
    )
despachantes.show()
despachantes.printSchema()

+---+-------------------+------+-------------+------+----------+
| id|               nome|status|       cidade|vendas|      data|
+---+-------------------+------+-------------+------+----------+
|  1|   Carminda Pestana| Ativo|  Santa Maria|    23|2020-08-11|
|  2|    Deolinda Vilela| Ativo|Novo Hamburgo|    34|2020-03-05|
|  3|   Emídio Dornelles| Ativo| Porto Alegre|    34|2020-02-05|
|  4|Felisbela Dornelles| Ativo| Porto Alegre|    36|2020-02-05|
|  5|     Graça Ornellas| Ativo| Porto Alegre|    12|2020-02-05|
|  6|   Matilde Rebouças| Ativo| Porto Alegre|    22|2019-01-05|
|  7|    Noêmia   Orriça| Ativo|  Santa Maria|    45|2019-10-05|
|  8|      Roque Vásquez| Ativo| Porto Alegre|    65|2020-03-05|
|  9|      Uriel Queiroz| Ativo| Porto Alegre|    54|2018-05-05|
| 10|   Viviana Sequeira| Ativo| Porto Alegre|     0|2020-09-05|
+---+-------------------+------+-------------+------+----------+

root
 |-- id: integer (nullable = true)
 |-- nome: string (nullable = true)
 |-- status: 

In [19]:
#Auto schema do spark
#Ler arquivo csv
#Arquivo sem nome de colunas
desp_autoSchema = spark.read.load(
    "/content/download/despachantes.csv",\
    header = False,\
    format = "csv",\
    sep = ",",\
    inferSchema = True
)
desp_autoSchema.show()
desp_autoSchema.printSchema()

+---+-------------------+-----+-------------+---+----------+
|_c0|                _c1|  _c2|          _c3|_c4|       _c5|
+---+-------------------+-----+-------------+---+----------+
|  1|   Carminda Pestana|Ativo|  Santa Maria| 23|2020-08-11|
|  2|    Deolinda Vilela|Ativo|Novo Hamburgo| 34|2020-03-05|
|  3|   Emídio Dornelles|Ativo| Porto Alegre| 34|2020-02-05|
|  4|Felisbela Dornelles|Ativo| Porto Alegre| 36|2020-02-05|
|  5|     Graça Ornellas|Ativo| Porto Alegre| 12|2020-02-05|
|  6|   Matilde Rebouças|Ativo| Porto Alegre| 22|2019-01-05|
|  7|    Noêmia   Orriça|Ativo|  Santa Maria| 45|2019-10-05|
|  8|      Roque Vásquez|Ativo| Porto Alegre| 65|2020-03-05|
|  9|      Uriel Queiroz|Ativo| Porto Alegre| 54|2018-05-05|
| 10|   Viviana Sequeira|Ativo| Porto Alegre|  0|2020-09-05|
+---+-------------------+-----+-------------+---+----------+

root
 |-- _c0: integer (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)

In [20]:
#Trabalhando com operadores
#<, >, &, |
despachantes.select('id',\
                    'nome',\
                    'vendas')\
                    .where(Fun.col('vendas')>20)\
                    .show()

despachantes.select('id',\
                    'nome',\
                    'vendas')\
                    .where((Fun.col('vendas')>20) & (Fun.col('vendas')<40))\
                    .show()

+---+-------------------+------+
| id|               nome|vendas|
+---+-------------------+------+
|  1|   Carminda Pestana|    23|
|  2|    Deolinda Vilela|    34|
|  3|   Emídio Dornelles|    34|
|  4|Felisbela Dornelles|    36|
|  6|   Matilde Rebouças|    22|
|  7|    Noêmia   Orriça|    45|
|  8|      Roque Vásquez|    65|
|  9|      Uriel Queiroz|    54|
+---+-------------------+------+

+---+-------------------+------+
| id|               nome|vendas|
+---+-------------------+------+
|  1|   Carminda Pestana|    23|
|  2|    Deolinda Vilela|    34|
|  3|   Emídio Dornelles|    34|
|  4|Felisbela Dornelles|    36|
|  6|   Matilde Rebouças|    22|
+---+-------------------+------+



In [21]:
despachantes.select('id', 'nome','vendas').where(Fun.col('vendas')>20).show()
despachantes.select('id', 'nome','vendas').where((Fun.col('vendas')>20) & (Fun.col('vendas')<40)).show()

+---+-------------------+------+
| id|               nome|vendas|
+---+-------------------+------+
|  1|   Carminda Pestana|    23|
|  2|    Deolinda Vilela|    34|
|  3|   Emídio Dornelles|    34|
|  4|Felisbela Dornelles|    36|
|  6|   Matilde Rebouças|    22|
|  7|    Noêmia   Orriça|    45|
|  8|      Roque Vásquez|    65|
|  9|      Uriel Queiroz|    54|
+---+-------------------+------+

+---+-------------------+------+
| id|               nome|vendas|
+---+-------------------+------+
|  1|   Carminda Pestana|    23|
|  2|    Deolinda Vilela|    34|
|  3|   Emídio Dornelles|    34|
|  4|Felisbela Dornelles|    36|
|  6|   Matilde Rebouças|    22|
+---+-------------------+------+



In [25]:
# Alterando nome das colunas do dataFrame 
# Vamos usar o comando withColumnRenamed('colunaAtua','novoNome')
# Atencao que printando o schema ele tambem sofre alteracao
despachantes2 = despachantes.withColumnRenamed('_c0', 'id')
despachantes2 = despachantes2.withColumnRenamed('_c1', 'nome')
despachantes2 = despachantes2.withColumnRenamed('_c2', 'status')
despachantes2 = despachantes2.withColumnRenamed('_c3', 'cidade')
despachantes2 = despachantes2.withColumnRenamed('_c4', 'vendas')
despachantes2 = despachantes2.withColumnRenamed('_c5', 'data')

despachantes2.columns
despachantes2.show()
despachantes2.printSchema()


+---+-------------------+------+-------------+------+----------+
| id|               nome|status|       cidade|vendas|      data|
+---+-------------------+------+-------------+------+----------+
|  1|   Carminda Pestana| Ativo|  Santa Maria|    23|2020-08-11|
|  2|    Deolinda Vilela| Ativo|Novo Hamburgo|    34|2020-03-05|
|  3|   Emídio Dornelles| Ativo| Porto Alegre|    34|2020-02-05|
|  4|Felisbela Dornelles| Ativo| Porto Alegre|    36|2020-02-05|
|  5|     Graça Ornellas| Ativo| Porto Alegre|    12|2020-02-05|
|  6|   Matilde Rebouças| Ativo| Porto Alegre|    22|2019-01-05|
|  7|    Noêmia   Orriça| Ativo|  Santa Maria|    45|2019-10-05|
|  8|      Roque Vásquez| Ativo| Porto Alegre|    65|2020-03-05|
|  9|      Uriel Queiroz| Ativo| Porto Alegre|    54|2018-05-05|
| 10|   Viviana Sequeira| Ativo| Porto Alegre|     0|2020-09-05|
+---+-------------------+------+-------------+------+----------+

root
 |-- id: integer (nullable = true)
 |-- nome: string (nullable = true)
 |-- status: 

In [26]:
# Alterando tipo string para date com o comando .withColumn passando como paramentro to_timestamp(Fun.col('coluna'), 'yyyy-MM-dd')
# convertendo para date  
despachantes2 = despachantes2.withColumn('data', to_timestamp(Fun.col('data'), 'yyyy-MM-dd'))
despachantes2.schema  

StructType(List(StructField(id,IntegerType,true),StructField(nome,StringType,true),StructField(status,StringType,true),StructField(cidade,StringType,true),StructField(vendas,IntegerType,true),StructField(data,TimestampType,true)))

In [27]:
# Praticando distinct
despachantes2.select(year('data')).distinct().show()

+----------+
|year(data)|
+----------+
|      2018|
|      2019|
|      2020|
+----------+



In [28]:
# Praticando orderBy
despachantes2.select('nome',year('data')).orderBy('nome').show()

+-------------------+----------+
|               nome|year(data)|
+-------------------+----------+
|   Carminda Pestana|      2020|
|    Deolinda Vilela|      2020|
|   Emídio Dornelles|      2020|
|Felisbela Dornelles|      2020|
|     Graça Ornellas|      2020|
|   Matilde Rebouças|      2019|
|    Noêmia   Orriça|      2019|
|      Roque Vásquez|      2020|
|      Uriel Queiroz|      2018|
|   Viviana Sequeira|      2020|
+-------------------+----------+



In [29]:
# Praticando groupBY com count
despachantes2.select('data').groupBy(year('data')).count().show()

+----------+-----+
|year(data)|count|
+----------+-----+
|      2018|    1|
|      2019|    2|
|      2020|    7|
+----------+-----+



In [30]:
# Praticando sum
despachantes2.select(Fun.sum('vendas')).show()

+-----------+
|sum(vendas)|
+-----------+
|        325|
+-----------+



In [35]:
despachantes2.orderBy(Fun.col('vendas').desc()).show()

+---+-------------------+------+-------------+------+-------------------+
| id|               nome|status|       cidade|vendas|               data|
+---+-------------------+------+-------------+------+-------------------+
|  8|      Roque Vásquez| Ativo| Porto Alegre|    65|2020-03-05 00:00:00|
|  9|      Uriel Queiroz| Ativo| Porto Alegre|    54|2018-05-05 00:00:00|
|  7|    Noêmia   Orriça| Ativo|  Santa Maria|    45|2019-10-05 00:00:00|
|  4|Felisbela Dornelles| Ativo| Porto Alegre|    36|2020-02-05 00:00:00|
|  2|    Deolinda Vilela| Ativo|Novo Hamburgo|    34|2020-03-05 00:00:00|
|  3|   Emídio Dornelles| Ativo| Porto Alegre|    34|2020-02-05 00:00:00|
|  1|   Carminda Pestana| Ativo|  Santa Maria|    23|2020-08-11 00:00:00|
|  6|   Matilde Rebouças| Ativo| Porto Alegre|    22|2019-01-05 00:00:00|
|  5|     Graça Ornellas| Ativo| Porto Alegre|    12|2020-02-05 00:00:00|
| 10|   Viviana Sequeira| Ativo| Porto Alegre|     0|2020-09-05 00:00:00|
+---+-------------------+------+------

In [39]:
#Tranalhando com orderBy
despachantes2.orderBy(Fun.col('nome').asc(),Fun.col('cidade').desc(), Fun.col('vendas').desc()).show()

+---+-------------------+------+-------------+------+-------------------+
| id|               nome|status|       cidade|vendas|               data|
+---+-------------------+------+-------------+------+-------------------+
|  1|   Carminda Pestana| Ativo|  Santa Maria|    23|2020-08-11 00:00:00|
|  2|    Deolinda Vilela| Ativo|Novo Hamburgo|    34|2020-03-05 00:00:00|
|  3|   Emídio Dornelles| Ativo| Porto Alegre|    34|2020-02-05 00:00:00|
|  4|Felisbela Dornelles| Ativo| Porto Alegre|    36|2020-02-05 00:00:00|
|  5|     Graça Ornellas| Ativo| Porto Alegre|    12|2020-02-05 00:00:00|
|  6|   Matilde Rebouças| Ativo| Porto Alegre|    22|2019-01-05 00:00:00|
|  7|    Noêmia   Orriça| Ativo|  Santa Maria|    45|2019-10-05 00:00:00|
|  8|      Roque Vásquez| Ativo| Porto Alegre|    65|2020-03-05 00:00:00|
|  9|      Uriel Queiroz| Ativo| Porto Alegre|    54|2018-05-05 00:00:00|
| 10|   Viviana Sequeira| Ativo| Porto Alegre|     0|2020-09-05 00:00:00|
+---+-------------------+------+------

In [40]:
despachantes2.groupBy('cidade').agg(sum('vendas')).show()

+-------------+-----------+
|       cidade|sum(vendas)|
+-------------+-----------+
|  Santa Maria|         68|
|Novo Hamburgo|         34|
| Porto Alegre|        223|
+-------------+-----------+



In [42]:
despachantes2.groupBy('cidade').agg(sum('vendas')).orderBy(Fun.col('sum(vendas)').desc()).show()

+-------------+-----------+
|       cidade|sum(vendas)|
+-------------+-----------+
| Porto Alegre|        223|
|  Santa Maria|         68|
|Novo Hamburgo|         34|
+-------------+-----------+



In [53]:
despachantes2\
.groupBy('cidade').agg(sum('vendas'))\
.orderBy(Fun.col('sum(vendas)').desc()).show()

+-------------+-----------+
|       cidade|sum(vendas)|
+-------------+-----------+
| Porto Alegre|        223|
|  Santa Maria|         68|
|Novo Hamburgo|         34|
+-------------+-----------+

